In [1]:
from scipy.spatial import distance
import editdistance
import random

In [2]:
'''
Initial chord sets.
'''
keys = ["C","Db","D","Eb","E","F","Gb","G","Ab","A","Bb","B"] #12 semi-tone

maj = dict()
maj7 = dict()
seventh = dict()
minor = dict()
min7 = dict()

for idx in keys:
    maj[idx+":maj"] = list()
    maj7[idx+":maj7"] = list()
    seventh[idx+":7"] = list()
    minor[idx+":min"] = list()
    min7[idx+":min7"] = list()

for root_key in keys:
    majchord = root_key+":maj"
    maj7chord = root_key+":maj7"
    seventhchord = root_key+":7"
    minorchord = root_key+":min"
    min7chord = root_key+":min7"

    rootIdx = keys.index(root_key)

    #maj
    maj[majchord].append(keys[rootIdx])
    maj[majchord].append(keys[(rootIdx+4)%12])
    maj[majchord].append(keys[(rootIdx+7)%12])

    #maj7
    maj7[maj7chord].append(keys[rootIdx])
    maj7[maj7chord].append(keys[(rootIdx+4)%12])
    maj7[maj7chord].append(keys[(rootIdx+7)%12])
    maj7[maj7chord].append(keys[(rootIdx+11)%12])

    #seventh
    seventh[seventhchord].append(keys[rootIdx])
    seventh[seventhchord].append(keys[(rootIdx+4)%12])
    seventh[seventhchord].append(keys[(rootIdx+7)%12])
    seventh[seventhchord].append(keys[(rootIdx+10)%12])

    #minor
    minor[minorchord].append(keys[rootIdx])
    minor[minorchord].append(keys[(rootIdx+3)%12])
    minor[minorchord].append(keys[(rootIdx+7)%12])

    #min7
    min7[min7chord].append(keys[rootIdx])
    min7[min7chord].append(keys[(rootIdx+3)%12])
    min7[min7chord].append(keys[(rootIdx+7)%12])
    min7[min7chord].append(keys[(rootIdx+10)%12])

'''
for Chord Histogram Initail
'''
Chord_Vector = list()
for each_key in keys:
    Chord_Vector.append(each_key+":maj")
    Chord_Vector.append(each_key+":min")
    Chord_Vector.append(each_key+":maj7")
    Chord_Vector.append(each_key+":min7")
    Chord_Vector.append(each_key+":7")



In [186]:
def get_chord_notes_set(chord):

    if "maj7" in chord:
        return maj7[chord]
    elif "min7" in chord:
        return min7[chord]
    elif "maj" in chord:
        return maj[chord]
    elif "min" in chord:
        return minor[chord]
    elif chord=='N' :
        return ['N','N','N']
    else:
        return seventh[chord]

In [102]:
strings = ['c', 'b', 'a']
strings.sort()
strings

['a', 'b', 'c']

In [206]:
def jaccard_similarity(x,y):
    '''
    Reference:
    Harte, C. (2010).
    Towards automatic extraction of harmony information from music signals
    (Doctoral dissertation, Department of Electronic Engineering, Queen Mary, University of London).
    '''
    
    x = get_chord_notes_set(x)
    y = get_chord_notes_set(y)
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    # I translate similarity to cost

    return (intersection_cardinality/float(union_cardinality))

In [107]:
a=get_chord_notes_set('D:maj')
a.sort()
a

['A', 'D', 'Gb']

In [152]:
get_chord_notes_set('N')

'N'

In [200]:
def LevenshteinDistcance(c1,c2):
    str1=get_chord_notes_set(c1)
    str2=get_chord_notes_set(c2)
    
    str1.sort()
    str2.sort()
    #print('c2:',c2,'str2:',str2)
    m = len(str1)
    n = len(str2)


    #Create a table to store results of subproblems 

    dp = [[MAX_COST for i in range(n+1)] for j in range(m+1)]
    # Fill d[][] in bottom up manner 
    for i in range(0,m+1):

        for j in range(0,n+1): 

            # If first string is empty, only option is to 
            # insert all characters of second string 
            if (i==0):
                dp[i][j] = j  # Min. operations = j 

            # If second string is empty, only option is to 
            # remove all characters of second string 
            elif(j==0):
                dp[i][j] = i # Min. operations = i 

            # If last characters are same, ignore last char 
            # and recur for remaining string 
            elif(str1[i-1] ==str2[j-1]): 
                dp[i][j] = dp[i-1][j-1]

            # If the last character is different, consider all 
            # possibilities and find the minimum 
            else:
                dp[i][j] = 1 + min(dp[i][j-1],  # Insert 
                                   dp[i-1][j],  # Remove 
                                   dp[i-1][j-1]) # Replace 


    
    return 1-dp[m][n]/max(m ,n)


In [198]:
LevenshteinDistcance('N','C:maj')

3


0.0

In [209]:
jaccard_similarity('C:maj','E:min')

0.5

In [210]:
LevenshteinDistcance('C:maj','E:min')

0.6666666666666667

In [8]:
x=get_chord_notes_set('C:maj')
y=get_chord_notes_set('G:maj')
intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
intersection_cardinality

1

In [43]:
def similarity(c1,c2,option):
    if(option=='jaccard_similarity'):
        return(jaccard_similarity(c1,c2))
    if(option=='LevenshteinDistcance'):
        return(LevenshteinDistcance(c1,c2))

In [41]:
def dtw(sa,sb,similarity_option):
    MAX_COST = 1<<32
    len_sa = len(sa)
    len_sb = len(sb)

    #Initial cost matrix
    dtw_array = [[MAX_COST for i in range(len_sa)] for j in range(len_sb)]

    #dtw_array[0][0] = distance(sa[0],sb[0])
    dtw_array[0][0] = jaccard_similarity(sa[0],sb[0])

    for i in range(0, len_sb):
        for j in range(0, len_sa):
            if i+j==0:
                continue
            nb = []
            if i > 0: nb.append(dtw_array[i-1][j])
            if j > 0: nb.append(dtw_array[i][j-1])
            if i > 0 and j > 0: nb.append(dtw_array[i-1][j-1])
            min_route = min(nb)
            #cost = distance(sa[j],sb[i])
            cost = similarity(sa[j],sb[i],similarity_option)
            dtw_array[i][j] = cost + min_route
    return dtw_array[len_sb-1][len_sa-1]

In [24]:
import pandas as pd
df = pd.read_pickle('../../../../../data/model_df_cleaned0609')

In [25]:
#擁有兩張譜以上的歌
#3517筆
has_2ver=df.groupby('song_name').size().loc[lambda x: x>1].keys()


In [34]:
# random.seed(613)
# trainsong_len=len(has_2ver)-1000 #有兩張以上的譜的歌數量減去1000首=訓練歌曲數量
# rd_song=random.sample(list(has_2ver),trainsong_len)
# test_song=has_2ver[~has_2ver.isin(rd_song)]
# df_test=df[df.song_name.isin(test_song)]
# df_test

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1                       _id  \
1              70            70             104  5cada0119fc5af34a460af1d   
2              71            71             105  5cada0119fc5af34a460af1f   
12             81            81             115  5cada0139fc5af34a460af3f   
13             82            82             116  5cada0149fc5af34a460af41   
25            104           104             138  5cada0169fc5af34a460af81   
26            105           105             139  5cada0169fc5af34a460af83   
27            106           106             140  5cada0169fc5af34a460af85   
28            107           107             141  5cada0169fc5af34a460af87   
45            129           129             164  5cada01a9fc5af34a460afcd   
46            130           130             165  5cada01a9fc5af34a460afcf   
47            131           131             166  5cada01a9fc5af34a460afd1   
52            138           138             173  5cada01b9fc5af34a460afe9   
53            139           139             174  5cada01b9fc5af34a460afeb   
74            167           167             202  5cada01f9fc5af34a460b035   
75            168           168             203  5cada01f9fc5af34a460b037   
76            169           169             204  5cada01f9fc5af34a460b039   
84            183           183             218  5cada0209fc5af34a460b057   
85            185           185             220  5cada0219fc5af34a460b05b   
86            186           186             221  5cada0219fc5af34a460b05d   
87            188           188             223  5cada0219fc5af34a460b061   
121           227           227             263  5cada0289fc5af34a460b0e5   
122           228           228             264  5cada0289fc5af34a460b0e9   
125           231           231             267  5cada0299fc5af34a460b0f1   
126           232           232             268  5cada0299fc5af34a460b0f3   
127           233           233             269  5cada0299fc5af34a460b0f5   
128           234           234             270  5cada0299fc5af34a460b0f7   
134           240           240             276  5cada02a9fc5af34a460b10d   
135           241           241             277  5cada02a9fc5af34a460b10f   
136           242           242             278  5cada02a9fc5af34a460b111   
137           243           243             279  5cada02a9fc5af34a460b113   
...           ...           ...             ...                       ...   
16829       19641         19641           21335  5cadaf2c9fc5af34a461a299   
16999       19826         19826           21557  5cadaf659fc5af34a461a53f   
17000       19827         19827           21558  5cadaf659fc5af34a461a541   
17005       19832         19832           21564  5cadaf669fc5af34a461a54f   
17006       19833         19833           21565  5cadaf669fc5af34a461a551   
17007       19834         19834           21566  5cadaf669fc5af34a461a553   
17031       19858         19858           21590  5cadaf6d9fc5af34a461a59d   
17032       19859         19859           21591  5cadaf6d9fc5af34a461a59f   
17046       19874         19874           21606  5cadaf729fc5af34a461a5cd   
17047       19875         19875           21607  5cadaf729fc5af34a461a5cf   
17061       19891         19891           21624  5cadaf769fc5af34a461a605   
17062       19892         19892           21625  5cadaf769fc5af34a461a607   
17063       19893         19893           21626  5cadaf779fc5af34a461a609   
17064       19894         19894           21627  5cadaf779fc5af34a461a60b   
17108       19945         19945           21684  5cadaf859fc5af34a461a6a6   
17109       19946         19946           21685  5cadaf859fc5af34a461a6a8   
17113       19953         19953           21693  5cadaf8a9fc5af34a461a6e8   
17114       19954         19954           21694  5cadaf8b9fc5af34a461a6ea   
17126       19969         19969           21711  5cadaf8f9fc5af34a461a722   
17127       19970         19970           21712  5cadaf8f9fc5af34a461a724   
171

In [26]:
#拿前1000首有兩張譜以上的歌出來當test songs
test_song=has_2ver[:1000]
df_test=df[df.song_name.isin(test_song)]

df_test

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1                       _id  \
7              76            76             110  5cada0129fc5af34a460af31   
8              77            77             111  5cada0129fc5af34a460af33   
12             81            81             115  5cada0139fc5af34a460af3f   
13             82            82             116  5cada0149fc5af34a460af41   
16             93            93             127  5cada0159fc5af34a460af63   
17             94            94             128  5cada0159fc5af34a460af65   
19             97            97             131  5cada0159fc5af34a460af6b   
20             98            98             132  5cada0169fc5af34a460af6d   
22            101           101             135  5cada0169fc5af34a460af79   
23            102           102             136  5cada0169fc5af34a460af7b   
24            103           103             137  5cada0169fc5af34a460af7d   
101           203           203             238  5cada0249fc5af34a460b089   
102           204           204             239  5cada0249fc5af34a460b08d   
103           205           205             240  5cada0249fc5af34a460b08f   
104           206           206             241  5cada0249fc5af34a460b091   
110           214           214             249  5cada0259fc5af34a460b0bb   
111           215           215             250  5cada0259fc5af34a460b0bf   
113           218           218             254  5cada0269fc5af34a460b0c7   
114           219           219             255  5cada0279fc5af34a460b0cb   
115           220           220             256  5cada0279fc5af34a460b0cd   
121           227           227             263  5cada0289fc5af34a460b0e5   
122           228           228             264  5cada0289fc5af34a460b0e9   
123           229           229             265  5cada0289fc5af34a460b0ed   
124           230           230             266  5cada0299fc5af34a460b0ef   
191           301           301             343  5cada0349fc5af34a460b1d9   
192           302           302             344  5cada0349fc5af34a460b1db   
193           303           303             345  5cada0349fc5af34a460b1dd   
194           304           304             346  5cada0349fc5af34a460b1df   
195           305           305             347  5cada0349fc5af34a460b1e1   
196           308           308             350  5cada0349fc5af34a460b1e7   
...           ...           ...             ...                       ...   
16820       19629         19629           21320  5cadaf289fc5af34a461a277   
16821       19630         19630           21321  5cadaf299fc5af34a461a27b   
16822       19631         19631           21322  5cadaf299fc5af34a461a27d   
16880       19697         19697           21395  5cadaf399fc5af34a461a349   
16881       19700         19700           21398  5cadaf399fc5af34a461a34f   
16882       19701         19701           21399  5cadaf3a9fc5af34a461a351   
16883       19703         19703           21401  5cadaf3b9fc5af34a461a357   
16884       19704         19704           21402  5cadaf3c9fc5af34a461a359   
16930       19756         19756           21486  5cadaf519fc5af34a461a455   
16931       19757         19757           21487  5cadaf519fc5af34a461a457   
16933       19759         19759           21489  5cadaf529fc5af34a461a465   
16934       19760         19760           21490  5cadaf529fc5af34a461a467   
16945       19772         19772           21502  5cadaf579fc5af34a461a49b   
16946       19773         19773           21503  5cadaf589fc5af34a461a49d   
16947       19774         19774           21504  5cadaf589fc5af34a461a49f   
16948       19775         19775           21505  5cadaf589fc5af34a461a4a1   
16949       19776         19776           21506  5cadaf589fc5af34a461a4a3   
17057       19885         19885           21617  5cadaf759fc5af34a461a5f5   
17058       19886         19886           21618  5cadaf759fc5af34a461a5f9   
17153       20000         20000           21743  5cadaf969fc5af34a461a76f   
171

In [33]:
std=df.iloc[0].chord
b0=df.iloc[0].song_chord
print(dtw(std, b0))

49.79999999999998


In [27]:
#df['tab_dtw_cost']=df.apply(lambda x: dtw(x['song_chord'],x['chord']), axis=1)

In [31]:
#df.to_pickle('../../../../../data/model_df_cleaned0609')

In [203]:
def dtw_table(x, y):
    nx = len(x)
    ny = len(y)
    table = np.zeros((nx+1, ny+1))
    
    # Compute left column separately, i.e. j=0.
    table[1:, 0] = np.inf
        
    # Compute top row separately, i.e. i=0.
    table[0, 1:] = np.inf
        
    # Fill in the rest.
    for i in range(1, nx+1):
        for j in range(1, ny+1):
            d = jaccard_similarity(x[i-1], y[j-1])
            table[i, j] = d + min(table[i-1, j], table[i, j-1], table[i-1, j-1])
    return table[i,j]

sa = ["C:maj","F:maj","G:maj","C:maj"]
sb = ["C:maj","F:maj","C:maj"] #delete C:maj
table=dtw_table(sa,sb)
table

0.8

In [35]:
#funtion:dtw
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    dtw_cost=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_cost.append(dtw(std,b0))
    min_dtw_cost_index=np.argmin(dtw_cost)
    if(min_dtw_cost_index==0):
        c_1 +=1
 

 

top1 = c_1 / float(1000)



print('Top-1 Precision: %f' % top1)




0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.427000


In [39]:
#similarity funtion:1-dtw(c1,c2,jaccard_similarity)/max(len(c1),len(c2)) 
#找similarity最高的譜
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    dtw_cost=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_cost.append(1-dtw(std,b0,'jaccard_similarity')/max(len(std),len(b0)))
    min_dtw_cost_index=np.argmax(dtw_cost)
    if(min_dtw_cost_index==0):
        c_1 +=1
 

 

top1 = c_1 / float(1000)



print('Top-1 Precision: %f' % top1)



0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.435000


In [201]:
#similarity funtion:1-DTW(cq1,cq2,LevenshteinDistcance)/max(len(cq1),len(cq2)) 
#找最高similarity的譜
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    dtw_cost=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_cost.append(1-dtw(std,b0,'LevenshteinDistcance')/max(len(std),len(b0)))
    min_dtw_cost_index=np.argmax(dtw_cost)
    if(min_dtw_cost_index==0):
        c_1 +=1
 

 

top1 = c_1 / float(1000)



print('Top-1 Precision: %f' % top1)



0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.421000


In [211]:
#funtion:dtw_table
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    dtw_cost=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_cost.append(dtw(std,b0,'jaccard_similarity'))
    min_dtw_cost_index=np.argmin(dtw_cost)
    if(min_dtw_cost_index==0):
        c_1 +=1
 

 

top1 = c_1 / float(1000)



print('Top-1 Precision: %f' % top1)


0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.409000
